In [ ]:
!pip install -q transformers datasets accelerate

import os, json, numpy as np
import torch
from datasets import load_dataset
from transformers import (
    T5Tokenizer, T5ForConditionalGeneration,
    Seq2SeqTrainingArguments, Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

# Silence wandb & HF hub chatter
os.environ["WANDB_DISABLED"] = "true"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.7 MB/s eta 0:00:00


In [ ]:
input_json_path = "/content/ingredient_data.json"   # <--- your uploaded file
jsonl_path      = "/content/ingredient_data.jsonl"

with open(input_json_path, "r") as f:
    raw_data = json.load(f)

with open(jsonl_path, "w") as f_out:
    for ex in raw_data:
        json.dump(
            {
                "input": ex["input"],
                "output": json.dumps(ex["output"])  # stringify dict
            },
            f_out
        )
        f_out.write("\n")

print(f"Converted {len(raw_data)} samples -> {jsonl_path}")

Converted 432 samples -> /content/ingredient_data.jsonl


In [ ]:
dataset = load_dataset("json", data_files=jsonl_path)["train"]
dataset = dataset.train_test_split(test_size=0.1, seed=42)
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 388
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 44
    })
})


In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

MAX_LEN = 96  # keep short to save memory

def preprocess(example):
    src = "parse: " + example["input"]
    tgt = example["output"]

    model_inputs = tokenizer(
        src,
        max_length=MAX_LEN,
        truncation=True,
        padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            tgt,
            max_length=MAX_LEN,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = dataset["train"].map(preprocess, remove_columns=dataset["train"].column_names)
tokenized_eval  = dataset["test"].map(preprocess,  remove_columns=dataset["test"].column_names)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/388 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/44 [00:00<?, ? examples/s]

In [ ]:
# ===============================
# 4) Data collator & metric
# ===============================
data_collator = DataCollatorForSeq2Seq(tokenizer, model=None)

def compute_metrics(eval_pred):
    preds, labels = eval_pred

    # Replace -100 with pad_token_id so we can decode
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds  = tokenizer.batch_decode(preds,   skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels,  skip_special_tokens=True)

    exact = [int(p.strip() == l.strip()) for p, l in zip(decoded_preds, decoded_labels)]
    return {"exact_match_accuracy": float(np.mean(exact))}

In [ ]:
# ===============================
# 5) Model & training args
# ===============================
model = T5ForConditionalGeneration.from_pretrained("t5-small")

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/t5-ingredient-parser",
    per_device_train_batch_size=4,        # smaller to reduce RAM
    per_device_eval_batch_size=4,
    num_train_epochs=7,
    logging_steps=25,
    eval_steps=None,
    save_strategy="epoch",
    eval_strategy="epoch",                # <-- use eval_strategy (your HF version expects this)
    learning_rate=5e-4,
    predict_with_generate=True,
    report_to="none",
    fp16=torch.cuda.is_available(),       # use mixed precision if GPU
    gradient_accumulation_steps=1
)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# ===============================
# 6) Trainer & Train
# ===============================
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

train_result = trainer.train()
trainer.save_model("/content/t5-ingredient-parser")
tokenizer.save_pretrained("/content/t5-ingredient-parser")

print("Training done & model saved.")

/tmp/ipython-input-7-2532000496.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Exact Match Accuracy
1,0.145800,0.072875,0.000000
2,0.052100,0.049996,0.000000
3,0.034100,0.044777,0.000000
4,0.027000,0.045965,0.000000
5,0.022500,0.044992,0.000000
6,0.014900,0.044775,0.000000
7,0.013900,0.045012,0.000000


Training done & model saved.


In [ ]:
metrics = trainer.evaluate()
print(f"\nFinal Exact Match Accuracy: {metrics['eval_exact_match_accuracy']*100:.2f}%")


Final Exact Match Accuracy: 0.00%


In [ ]:
def parse_ingredient_t5(text):
    inp = "parse: " + text
    inputs = tokenizer([inp], return_tensors="pt", truncation=True).to(model.device)
    output_ids = model.generate(**inputs, max_length=64)
    decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    try:
        return json.loads(decoded)
    except json.JSONDecodeError:
        print("Invalid JSON:", decoded)
        return {}

# Quick tests
print(parse_ingredient_t5("2 ½ tablespoons olive oil"))
print(parse_ingredient_t5("Add 1 pinch of cinnamon"))

Invalid JSON: "quantity": "0.25", "unit": "tablespoons", "ingredient": "olive oil"
{}
Invalid JSON: "quantity": "1", "unit": "pinch", "ingredient": "cinnamon"
{}


In [ ]:
import shutil

shutil.make_archive("/content/t5-ingredient-parser", 'zip', "/content/t5-ingredient-parser")

'/content/t5-ingredient-parser.zip'